In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dateutil
from sklearn import ensemble,metrics,linear_model,tree

# Read All data

In [2]:
campData=pd.read_csv('campaign_data.csv')
coupon_item_mappingData=pd.read_csv('coupon_item_mapping.csv')
customer_demographicsData=pd.read_csv('customer_demographics.csv')
c_trans_Data=pd.read_csv('customer_transaction_data.csv')
item_dataData=pd.read_csv('item_data.csv')
trainData=pd.read_csv('train.csv')
test_QyjYwdjData=pd.read_csv('test_QyjYwdj.csv')

In [4]:
trainData['dtType']='train'
test_QyjYwdjData['dtType']='test'

allData=trainData.append(test_QyjYwdjData,sort=False)
allData.shape

(128595, 6)

In [ ]:
# customer_transaction_dataData['TransactionAmt_to_mean_card1'] = train['selling_price'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')

In [5]:
allData.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType
0,1,13,27,1053,0.0,train
1,2,13,116,48,0.0,train


## feature in  Camp Data

In [ ]:
campData['start_date']=campData['start_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['end_date']=campData['end_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['duration']=campData['end_date']-campData['start_date']
campData['duration']=campData['duration'].apply(lambda x: x.days)
campData['start_Month']=campData['start_date'].apply(lambda x: x.month)
campData['end_Month']=campData['end_date'].apply(lambda x: x.month)

In [ ]:
campData.head()

In [ ]:
#customer previous campaign reaction
# item data transaction data merged4
# train data and camp data merged

## Merge transaction data and Item data

In [ ]:
transItemData=pd.merge(c_trans_Data,item_dataData,on='item_id',how='left')
transItemData.shape

transItemData['coupon_discountAvail']=transItemData['coupon_discount'].apply(lambda x: 1 if x !=0 else 0)

In [ ]:
transItemData.head(2)

In [ ]:
itemCouponDetail=transItemData.groupby('item_id',as_index=False).agg({'coupon_discountAvail':{'sumOFCoupon':'sum','totalCount':'count'}})
itemCouponDetail.columns=[''.join(jj) for jj in itemCouponDetail.columns]
itemCouponDetail['probaItem']=itemCouponDetail['coupon_discountAvailsumOFCoupon']/itemCouponDetail['coupon_discountAvailtotalCount']
# itemCouponDetail[itemCouponDetail['probaItem']>0]

# Merge traindata with Campdata

In [ ]:
trainCampDatamerge=pd.merge(trainData,campData,on='campaign_id',how='left')
trainCampDatamerge.shape,trainData.shape

In [ ]:
trainCampDatamerge.head(2)

## Feature engg in coupon mapping data and item data and transaction data

### merge mapping dtaa and transaction data

In [ ]:
itenACoupData=pd.merge(coupon_item_mappingData,item_dataData,on='item_id',how='left').sort_values('coupon_id')

In [ ]:
itenACoupData.head(2)

### feature from transaction data Item level

In [ ]:
itemInfoFromTransaction=c_trans_Data.groupby('item_id',as_index=False).agg({'selling_price':'mean',
                                                                     'coupon_discount':{'discountCount':'count','totalDisc':'sum'},
                                                                    'other_discount':{'OTdiscountCount':'count','OTtotalDisc':'sum'}})
itemInfoFromTransaction.columns=[''.join(j) for j in itemInfoFromTransaction.columns]

In [ ]:
itemInfoFromTransaction['dicountPerGiven']=abs(itemInfoFromTransaction['coupon_discounttotalDisc']/itemInfoFromTransaction['coupon_discountdiscountCount'])
itemInfoFromTransaction['dicountOthPerGiven']=abs(itemInfoFromTransaction['other_discountOTtotalDisc']/itemInfoFromTransaction['other_discountOTdiscountCount'])
itemInfoFromTransaction['dicountGivenPerc']=abs(itemInfoFromTransaction['dicountPerGiven']/itemInfoFromTransaction['selling_pricemean'])
itemInfoFromTransaction['dicountOthPerGivenPerc']=abs(itemInfoFromTransaction['dicountOthPerGiven']/itemInfoFromTransaction['selling_pricemean'])


In [ ]:
itemInfoFromTransaction.head(2)

In [ ]:
itenACoupData.shape,itemInfoFromTransaction.shape,coupon_item_mappingData.shape,item_dataData.shape

In [ ]:
itenACoupData.head(2)

### merge item information from transaction to coupon item information

In [ ]:
mergedItemCoupon=pd.merge(itenACoupData,itemInfoFromTransaction,on='item_id',how='left').fillna(0)
mergedItemCoupon=pd.merge(mergedItemCoupon,itemCouponDetail[['item_id','probaItem']],on='item_id',how='left').fillna(0)
#some  null values present

In [ ]:
mergedItemCoupon.head()

In [ ]:
coupon_categFind=mergedItemCoupon.groupby(['coupon_id','category','brand','brand_type'],as_index=False).count()[['coupon_id','category','brand','brand_type','item_id']]

In [ ]:
couponProbItemCoup=mergedItemCoupon.groupby('coupon_id',as_index=False).agg({'probaItem':'mean'})

In [ ]:
coupon_categFind=coupon_categFind.sort_values(['coupon_id','item_id'],ascending=False)
coupon_categFind=coupon_categFind.drop_duplicates('coupon_id')

In [ ]:
coupon_categFind=pd.merge(coupon_categFind,couponProbItemCoup,on='coupon_id',how='left')

In [ ]:
del coupon_categFind['item_id']

In [ ]:
coupon_categFind.head()

In [ ]:
mergedItemCoupon[mergedItemCoupon['selling_pricemean'].isnull()]

## Feature from customer data

In [ ]:
customerProfileData=c_trans_Data.groupby('customer_id',as_index=False).agg({'quantity':'sum','date':'nunique','item_id':'count',
                                                                        'selling_price':{'maxSP':'max','minSP':'min',
                                                                                        'averageSP':'mean'},
                                                                        'other_discount':{'maxOD':'max','minOD':'min',
                                                                                        'averageOD':'mean'},
                                                                        'coupon_discount':{'maxCD':'max','minCD':'min',
                                                                                        'averageCD':'mean','totalCDAvailed':'count'}})

customerProfileData.columns = ['_'.join(col) for col in customerProfileData.columns.values]

In [ ]:
customerProfileData.head(2)

### Clustering on  transaction item data information to get information about coupons

In [ ]:
mergedItemCoupon.head(2)

In [ ]:
datatoCluster=mergedItemCoupon.groupby('coupon_id',as_index=False).mean()[['coupon_id', 
       'selling_pricemean', 'coupon_discountdiscountCount',
       'coupon_discounttotalDisc', 'other_discountOTdiscountCount',
       'other_discountOTtotalDisc', 'dicountPerGiven', 'dicountOthPerGiven',
       'dicountGivenPerc', 'dicountOthPerGivenPerc','probaItem']]

clusVar=['coupon_id', 
       'selling_pricemean', 'coupon_discountdiscountCount',
       'coupon_discounttotalDisc', 'other_discountOTdiscountCount',
       'other_discountOTtotalDisc', 'dicountPerGiven', 'dicountOthPerGiven',
       'dicountGivenPerc', 'dicountOthPerGivenPerc','probaItem']

from sklearn import preprocessing,cluster,decomposition

preProCl=preprocessing.StandardScaler()
clusData=preProCl.fit_transform(datatoCluster[clusVar])

pcaComp=decomposition.PCA()
pcaData=pcaComp.fit_transform(clusData)
pcaComp.explained_variance_ratio_

In [ ]:
pcaDataFToGraph=pd.DataFrame(pcaData[:,:2],columns=['pca1','pca2'])
kM=cluster.KMeans()
pcaDataFToGraph['clusCat']=kM.fit_predict(clusData[:,:5])
pcaDataFToGraph.head(2)
sns.scatterplot(pcaDataFToGraph['pca1'],pcaDataFToGraph['pca2'],hue=pcaDataFToGraph['clusCat'])

In [ ]:
# pcaDataFToGraph['clusCat']=pcaDataFToGraph['clusCat'].map(str)

In [ ]:
pcaDataTOCoup=pd.concat([datatoCluster[['coupon_id']],pcaDataFToGraph],axis=1)

In [ ]:
datatoCluster.head(2)

In [ ]:
# crosstabData=pd.pivot_table(coupon_item_mappingData,index='coupon_id',columns='item_id',values='val',aggfunc=np.sum).fillna(0)
# coupon_idData=trainCampDatamerge.groupby('coupon_id',as_index=False).agg({'redemption_status':'sum'})
# coupon_idData[coupon_idData['redemption_status']>1]

# Final data for model prep

In [ ]:
del datatoCluster['probaItem']

In [ ]:
X1=pd.merge(allData,coupon_categFind,on='coupon_id',how='left')
X2=pd.merge(X1,datatoCluster,on='coupon_id',how='left')
X3=pd.merge(X2,pcaDataTOCoup,on='coupon_id',how='left')
X4=pd.merge(X3,campData,on='campaign_id',how='left')
X5=pd.merge(X4,customer_demographicsData,on='customer_id',how='left')
X6=pd.merge(X5,customerProfileData,left_on='customer_id',right_on='customer_id_',how='left')
# X2.head(2)

In [ ]:
X5['marital_status']=X5['marital_status'].fillna('Other')
X5['rented']=X5['rented'].fillna(0)
X5['no_of_children']=X5['no_of_children'].fillna('0')
X5['age_range']=X5['age_range'].fillna('Other')
X5['family_size']=X5['family_size'].fillna('9999')
X5['income_bracket']=X5['income_bracket'].fillna(9999)

In [ ]:
llP=preprocessing.LabelEncoder()
# X5[jj]=llP.fit_transform(X5['campaign_type'])

In [ ]:
X5[['category', 'clusCat', 'campaign_type', 'no_of_children', 'income_bracket']].head(2)

In [ ]:
# X5['no_of_children']=X5['no_of_children'].map(str)

In [ ]:
pd.unique(X5['family_size'])

In [ ]:
categoColumns=[ 'campaign_type','no_of_children','category','brand_type','age_range','marital_status','family_size']
# catData=pd.get_dummies(X5[categoColumns],prefix_sep='_',)
for jj in categoColumns:
    print (jj)
    llP=preprocessing.LabelEncoder()
    X5[jj]=llP.fit_transform(X5[jj])

In [ ]:
# newtData=pd.concat([X5,catData],sort=False,axis=1)
newtData=X5

In [ ]:
XVar=list(newtData.columns)
XVar.remove('id')
XVar.remove('redemption_status')
XVar.remove('dtType')
XVar.remove('start_date')
XVar.remove('end_date')
YVar='redemption_status'

In [ ]:
# for j in categoColumns+['campaign_id', 'coupon_id', 'customer_id']:
#     XVar.remove(j)

In [ ]:
trainPart=newtData[newtData['dtType']=='train']
testPart=newtData[newtData['dtType']=='test']
trainPart.shape,testPart.shape

In [ ]:
XVar

In [ ]:
from sklearn import model_selection,metrics
import xgboost as xgb

## Train test Split Train, Val, Test set

In [ ]:
trainX,testHX,trainY,testHY=model_selection.train_test_split(trainPart[XVar],trainPart[YVar],test_size=.2,stratify =trainPart[YVar])
trainX.shape,testHX.shape,trainY.shape,testHY.shape

In [ ]:
trainX1,valX1,trainY1,valY1=model_selection.train_test_split(trainX,trainY,test_size=.2,stratify =trainY)
trainX1.shape,valX1.shape,trainY1.shape,valY1.shape

In [ ]:
modelR=xgb.XGBClassifier().fit(trainX1,trainY1)

In [ ]:
t1=modelR.predict(trainX)
t2=modelR.predict(valX1)
t3=modelR.predict(testHX)
trainScore=modelR.predict_proba(trainX)[:,1]
testScore=modelR.predict_proba(valX1)[:,1]
test2Score=modelR.predict_proba(testHX)[:,1]

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY,trainScore),metrics.roc_auc_score(valY1,testScore),metrics.roc_auc_score(testHY,test2Score)

In [ ]:
import lightgbm  as lgb

In [ ]:
modelL=lgb.LGBMClassifier().fit(trainX1,trainY1)

In [ ]:
t1=modelL.predict(trainX)
t2=modelL.predict(valX1)
t3=modelL.predict(testHX)
trainScore=modelL.predict_proba(trainX)[:,1]
testScore=modelL.predict_proba(valX1)[:,1]
test2Score=modelL.predict_proba(testHX)[:,1]

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY,trainScore),metrics.roc_auc_score(valY1,testScore),metrics.roc_auc_score(testHY,test2Score)

In [ ]:
modelRF=ensemble.RandomForestClassifier().fit(trainX1,trainY1)

In [ ]:
t1=modelRF.predict(trainX)
t2=modelRF.predict(valX1)
t3=modelRF.predict(testHX)
trainScore=modelRF.predict_proba(trainX)[:,1]
testScore=modelRF.predict_proba(valX1)[:,1]
test2Score=modelRF.predict_proba(testHX)[:,1]

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY,trainScore),metrics.roc_auc_score(valY1,testScore),metrics.roc_auc_score(testHY,test2Score)

In [ ]:
params={'base_score':0.9,
        'booster':'gbtree', 
        'colsample_bylevel':1,
       'colsample_bytree':1, 
        'gamma':0, 
        'learning_rate':0.1, 
        'max_delta_step':0,
       'max_depth':3, 
        'min_child_weight':1, 
        'n_estimators':100,
        'eval_metric': ['logloss','auc'],
       'n_jobs':1,  'objective':'binary:logistic', 'random_state':0,
       'reg_alpha':0, 'reg_lambda':1, 'scale_pos_weight':1, 'subsample':1,
       'early_stopping_rounds' : 50,
       'stratified':True }

In [ ]:
num_round = 1000

dtrain = xgb.DMatrix(trainX1,trainY1)
# dtrain.set_group(Xg)

dvalid = xgb.DMatrix(valX1, valY1)
# dvalid.set_group(Xgt)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
bst = xgb.train(params, dtrain, num_round, evals = watchlist,
    early_stopping_rounds=50,verbose_eval=50,maximize=True)


In [ ]:
dtest = xgb.DMatrix(testHX)

In [ ]:
testScoreP=bst.predict(dtest)

In [ ]:
metrics.roc_auc_score(testHY,testScoreP)

In [ ]:
dtestOrg = xgb.DMatrix(testPart[XVar])
testScoreP2=bst.predict(dtestOrg)

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:testScoreP2})
firstSub2.to_csv('firstSub8.csv',index=False)